In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras

In [26]:
import os

# Customised for Andy's setup
# os.chdir('SharedFiles/MAST30034-Group-Project/')

In [27]:
# labels_path = os.join()

# Customised for Kia's setup
labels = pd.read_csv("kaggle_bee_vs_wasp/labels.csv")
labels['path'] = labels['path'].str.replace("\\", "/")

By removing the low quality photos, we get rid of the is_other instances

In [28]:
high_quality_entries = labels[labels["photo_quality"] == 1]

In [29]:
print(high_quality_entries.shape)
high_quality_entries.head()

(7035, 10)


,id,path,is_bee,is_wasp,is_otherinsect,is_other,photo_quality,is_validation,is_final_validation,label
0,1,bee1/10007154554_026417cfd0_n.jpg,1,0,0,0,1,0,0,bee
1,2,bee1/10024864894_6dc54d4b34_n.jpg,1,0,0,0,1,0,1,bee
2,3,bee1/10092043833_7306dfd1f0_n.jpg,1,0,0,0,1,1,0,bee
3,4,bee1/1011948979_fc3637e779_w.jpg,1,0,0,0,1,0,1,bee
4,5,bee1/10128235063_dca17db76c_n.jpg,1,0,0,0,1,0,0,bee


In [30]:
np.unique(high_quality_entries["label"], return_counts = True)

(array(['bee', 'insect', 'wasp'], dtype=object), array([2469, 2439, 2127]))

In [31]:
# no non-insects found in the dataset
list(high_quality_entries["is_other"]).count(1)

0

# Import Data

In [32]:
from matplotlib.image import imread

In [33]:
images = []

# random_sample = high_quality_entries.sample(10)
# Train 50 for now just to get the code running
for image_path in images_train_path:
    #image_path = image_path.replace("\\", "/")
    images.append(imread(os.path.join("kaggle_bee_vs_wasp", image_path[0])))

# Preprocessing

In [38]:
from skimage.transform import resize

In [57]:
# resize images to 224 x 224 
images_to_remove = []
for im in range(len(images)):
    images[im] = resize(images[im], (224,224))

    #only keep coloured images and only three channels of each
    if ((np.ndim(images[im])<3) or (images[im].shape[2]<3)):
        images_to_remove.append(im)
    elif images[im].shape[2]>3:
        images[im] = images[im][:,:,0:3]




In [63]:
for index in images_to_remove:
    del images[index]

The images are supposed to have 3 channels - RGB

There are 2 anomalous images.
Image 2538 ('wasp1/50988238_1c313384d9_n.jpg') is black and white (1-channel)

Image 4163 ('bee1/509769820_83fa0a3ea9_n.jpg') has an extra channel for transparency, which can be removed.

In [ ]:
# #remove the 4th channel for the other
# images[4163] = images[4163][:,:,0:3]
    
# # Delete the black and white one,
# del images[2538]
# label_train = np.delete(label_train, [2538])

In [ ]:
# f, ax = plt.subplots(1,3)

# # left: Original
# ax[0].imshow(images[4163])

# # middle: Transparency removed, still the same
# ax[1].imshow(images[4163][:,:,0:3])

# # right: This is how the original looks like in the file.
# # I had to subtract and add greys to reproduce it here for convenience.
# ax[2].imshow(-images[2538], cmap='Greys')

# Split the Data

In [65]:
from sklearn.model_selection import train_test_split

images_path = high_quality_entries["path"].values.reshape(-1, 1)
labels = high_quality_entries["label"].values.reshape(-1, 1)
images_train_path, images_test_path, label_train, label_test = train_test_split(images_path, labels, test_size = 0.3, random_state = 30034, stratify = labels)

# Prepare for model

In [66]:
from sklearn.preprocessing import LabelEncoder

In [67]:
encoder = LabelEncoder()
transformed_label = encoder.fit_transform(np.ravel(label_train))
print(transformed_label[0:6])

[2 2 0 1 0 2]


In [68]:
images = np.array(images)
inputs = keras.applications.resnet50.preprocess_input(images) 

In [69]:
inputs.shape

(4923, 224, 224, 3)

In [70]:
label_train.shape

(4924, 1)

# Model 1 - ResNet50

In [71]:
model = keras.applications.resnet50.ResNet50(weights="imagenet")

In [72]:
Y_proba = model.predict(inputs)

KeyboardInterrupt: 

In [ ]:
# top_K = keras.applications.resnet50.decode_predictions(Y_proba, top=2)
# for image_index in range(len(images)):
#     print("Image #{}".format(image_index))

#     for class_id, name, y_proba in top_K[image_index]:
#         print(" {} - {:12s} {:.2f}%".format(class_id, name, y_proba * 100))
#     print()

# Model 2

We use ResNet50 as a base model. The final layer is not adapated to our problem (wrong classes), so we remove that layer, add our own layers and our own output.

ResNet50 performs well. We do not want to adjust the pre-trained layers, which should be good at detecting low-level details.

In [ ]:
model2_base = keras.applications.resnet50.ResNet50(weights="imagenet",
                                             include_top = False)
avg = keras.layers.GlobalAveragePooling2D()(model2_base.output)
output = keras.layers.Dense(3, activation='softmax')(avg)

model2_final = keras.Model(inputs = model2_base.input, outputs = output)

for layer in model2_base.layers:
    layer.trainable = False

optimizer = keras.optimizers.SGD(lr=0.2, momentum= 0.9, decay = 0.01)

model2_final.compile(loss="sparse_categorical_crossentropy", 
                     optimizer=optimizer,
                     metrics=["accuracy"])

In [ ]:
type(label_train)

In [ ]:
inputs.shape

In [ ]:
history = model2_final.fit(x=inputs, y=transformed_label, 
                           epochs=5, validation_split=0.2)